In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/tweets-sg-100/tweets_sg_100.trainables.syn1neg.npy
/kaggle/input/tweets-sg-100/tweets_sg_100
/kaggle/input/tweets-sg-100/tweets_sg_100.wv.vectors.npy
/kaggle/input/partiality-scores/labeled.csv
/kaggle/input/partiality-scores/label.csv


In [3]:
from nltk.stem.isri import ISRIStemmer
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import gensim
import re


df = pd.read_csv("/kaggle/input/partiality-scores/labeled.csv")
sentences = df["preprocessed tweets"].astype(str)

tweets_sg_100 = '/kaggle/input/tweets-sg-100/tweets_sg_100'
sg_model = gensim.models.Word2Vec.load(tweets_sg_100)
# ANALYZE HASHTAGS

In [6]:
df.head()

,tweet_id,tweet_text,partiality_score,preprocessed tweets,label
0,1641953510903234562,روس تحتج علي تصرف استفزازيه قوات امريكيه في سور,4.50,روس تحتج عل تصرف استفزاز قوا امريك في سور,neutral
1,1641953254094381057,روس تحتج علي استفزاز قوات اميركيه في سور,4.50,روس تحتج عل استفزاز قوا اميرك في سور,neutral
2,1641953209810862080,توضيح لن يحدث شي مهم هذا اسبوع ومن متوقع تبدا ...,-5.25,توضيح لن يحدث شي مهم هذا اسبوع ومن متوقع تبد ا...,pro russia
3,1641953161295347716,فرنسا بتشوف ايام شرق اوسطيه افريقيه زي الفل يع...,-5.25,رنس تشوف ايام شرق اوسط افريق زي الفل يعن رييس ...,pro russia
4,1641952955136974848,روس تحتج علي استفزاز قوات اميركيه في سور سكاي ...,4.50,روس تحتج عل استفزاز قوا اميرك في سور سكا نيوز عرب,neutral


In [ ]:
# adding important words to the embedder, words he didnt know about

# important words not recognized by the embedder 
russia = ['روسي','روس','روسيه','لروسي','والروس','ياروسي','روسی','موسك'] # 'روسيا'
belarus = ['يلاروسي','يلاروس','بيلاروس','بيلاروسيه','لبيلاروسي','والبيلاروس'] # 'بيلاروسيا'
ukr = ['اوكران','واوكراني','اوكر','لاوكراني','ناتو'] # 'اوكرانيا'
russian_hashtags = ['جيشالروس','قواتالروسيه','روسياليوم','روسياتنتصر','خارجيهالروسيه','عمليهالعسكريهالروسيه'] # روسيا
ukr_russian_hashtags = ['حربالروسيهالاوكرانيه','روسياوكراني','اوكرانياروسي','غزوالروسيلاوكرانيا','روسياواوكراني'] # 'حرب'
ukr_hashtags = ['روسياتغزواوكراني','روسياتكذب','روسيالص','روسياوالص','روسيامجرم','مجدلاوكرانيا','مخابراتالاوكرانيه'] # 'اوكرانيا'
zelensky = ['زيلينسك','زلينسك'] # 'اوباما'
putin = ['بوتن','وتن','پوتن','وتين','بوتين','پوتين'] #'بوتين'
war = ['حربالعالميه', 'حربالعالميهالثالثه','عسكريه'] # 'حرب'
egypt = ['مصر','مصريه','لمصر'] # 'عربي'
us = ['وامريك','الولاياتالمتحده','الولاياتالمتحده','الولاياتالمتحدهالامريكيه','البريطانيا','والولايا','امريك'] # 'امريكا'
gb = ['ريطاني'] # "بريطانيا"

russia.extend(russian_hashtags)
ukr.extend(ukr_hashtags)
war.extend(ukr_russian_hashtags)

added_words = []
added_words.extend(russia)
added_words.extend(ukr)
added_words.extend(war)
added_words.extend(putin)
added_words.extend(zelensky)
added_words.extend(egypt)
added_words.extend(us)
added_words.extend(gb)
added_words = set(added_words)
russia = set(russia)
ukr = set(ukr)
war = set(war)
putin = set(putin)
zelensky = set(zelensky)
egypt = set(egypt)
us = set(us)
gb = set(gb)

# add words that donot exist to wv
# russia 'روسيا', ukr 'اوكرانيا', war 'حرب' , putin 'بوتين', zelensky 'اوباما', belarus 'بيلاروسيا' , egypt 'قطر', us 'امريكا', gb "بريطانيا"
for word in russia:
    sg_model.wv.add_vector(word,sg_model.wv['روسيا'])
for word in ukr:
    sg_model.wv.add_vector(word,sg_model.wv['اوكرانيا'])
for word in war:
    sg_model.wv.add_vector(word,sg_model.wv['حرب'])
for word in putin:
    sg_model.wv.add_vector(word,sg_model.wv['بوتين'])
for word in zelensky:
    sg_model.wv.add_vector(word,sg_model.wv['اوباما'])
for word in belarus:
    sg_model.wv.add_vector(word,sg_model.wv['بيلاروسيا'])
for word in egypt:
    sg_model.wv.add_vector(word,sg_model.wv['قطر'])   
for word in us:
    sg_model.wv.add_vector(word,sg_model.wv['امريكا']) 
for word in gb:
    sg_model.wv.add_vector(word,sg_model.wv["بريطانيا"])

In [66]:
ex_length = 100000
max_len = max([ len(sentence.split()) for sentence in sentences ])
encoder_input_data = np.zeros([ex_length,max_len+5,100])


In [46]:
# russia 'روسيا', ukr 'اوكرانيا', war 'حرب' , putin 'بوتين', zelensky 'اوباما', belarus 'بيلاروسيا' , egypt 'قطر', us 'امريكا', gb "بريطانيا"
for ex in range(100000,100000+ex_length):
    words = sentences[ex].split()
    position = 0
    for word in words:
        if word in sg_model.wv:
            encoder_input_data[ex-100000,position,:] = sg_model.wv[word]
        position += 1

In [45]:
df['label'].shape

(219406,)

In [47]:

# labels are in the third column
y = df['label'][100000:200000]

x = encoder_input_data



In [48]:
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

# Map labels to integer values
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y)

# Example output:
# y_train_encoded = [1, 0, 2, 1, 0]

# Perform one-hot encoding
y_train_one_hot = to_categorical(y_train_encoded)

In [49]:
x.shape

(100000, 64, 100)

In [50]:
y.shape

(100000,)

In [51]:
y_train_one_hot[0]

array([0., 0., 1.])

In [ ]:
df = pd.read_csv('/kaggle/input/partiality-scores/unlabeled_tweets.csv')
unlabeled_features = df['tweets']

In [ ]:
ex_length = len(unlabeled_features)
max_len = max([ len(sentence.split()) for sentence in sentences ])
X_unlabeled = np.zeros([ex_length,max_len+5,100])

In [ ]:
# russia 'روسيا', ukr 'اوكرانيا', war 'حرب' , putin 'بوتين', zelensky 'اوباما', belarus 'بيلاروسيا' , egypt 'قطر', us 'امريكا', gb "بريطانيا"
for ex in range(ex_length):
    words = unlabeled_features[ex].split()
    position = 0
    for word in words:
        if word in sg_model.wv:
            X_unlabeled[ex,position,:] = sg_model.wv[word]
        position += 1

In [55]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Bidirectional, LSTM, Dense
from tensorflow.keras.losses import CategoricalCrossentropy

# Define the model
model = Sequential()

# Add Bidirectional LSTM layers
model.add(Bidirectional(LSTM(units=64, return_sequences=True), input_shape=(None, 100)))
model.add(Bidirectional(LSTM(units=128, return_sequences=False)))

# Add Dense layer
model.add(Dense(units=20, activation='relu'))

# Add Softmax layer
model.add(Dense(units=3, activation='softmax'))

loss= CategoricalCrossentropy(
    from_logits=False,
    label_smoothing=0.0,
    axis=-1,
    reduction="sum_over_batch_size",
    name="categorical_crossentropy",
)
# Compile the model
model.compile(optimizer='adam', loss=loss, metrics=['accuracy'])

# Print model summary
model.summary()


Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ bidirectional_8 (Bidirectional) │ (None, None, 128)      │        84,480 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_9 (Bidirectional) │ (None, 256)            │       263,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 20)             │         5,140 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 3)              │            63 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 352,851 (1.35 MB)

 Trainable params: 352,851 (1.35 MB)

 Non-trainable params: 0 (0.00 B)

In [57]:
model.load_weights('best_weights.keras')

/opt/conda/lib/python3.10/site-packages/keras/src/saving/saving_lib.py:418: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 34 variables. 
  trackable.load_own_variables(weights_store.get(inner_path))


In [60]:
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint filepath with '.keras' extension
checkpoint_filepath = 'best_weights.keras'

# Define the ModelCheckpoint callback
checkpoint_callback = ModelCheckpoint(filepath=checkpoint_filepath,
                                      save_best_only=True,
                                      monitor='val_accuracy',
                                      mode='max',  # 'max' for accuracy
                                      verbose=1)

# Train the model with ModelCheckpoint callback
history = model.fit(x, y_train_one_hot,
                    epochs=3,
                    batch_size=32,
                    validation_split=0.1,
                    callbacks=[checkpoint_callback])


Epoch 1/3
2812/2813 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.9795 - loss: 0.0561
Epoch 1: val_accuracy improved from -inf to 0.93180, saving model to best_weights.keras
2813/2813 ━━━━━━━━━━━━━━━━━━━━ 53s 19ms/step - accuracy: 0.9795 - loss: 0.0561 - val_accuracy: 0.9318 - val_loss: 0.2539
Epoch 2/3
2812/2813 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.9830 - loss: 0.0467
Epoch 2: val_accuracy did not improve from 0.93180
2813/2813 ━━━━━━━━━━━━━━━━━━━━ 52s 18ms/step - accuracy: 0.9830 - loss: 0.0467 - val_accuracy: 0.9268 - val_loss: 0.2894
Epoch 3/3
2811/2813 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.9884 - loss: 0.0323
Epoch 3: val_accuracy did not improve from 0.93180
2813/2813 ━━━━━━━━━━━━━━━━━━━━ 52s 18ms/step - accuracy: 0.9884 - loss: 0.0323 - val_accuracy: 0.9242 - val_loss: 0.3459


In [ ]:
# the substitute vector if we dropped the word
drop_symbol = np.zeros(100)
consistency_lambda = 0.01
loss_fn = CategoricalCrossentropy(from_logits=False,label_smoothing=0.0,axis=-1,reduction="sum_over_batch_size",name="categorical_crossentropy")
# Compile the model
model.compile(optimizer=optimizer, loss=loss_fn, metrics=['accuracy'])

# Training loop
for epoch in range(10):
    # Train model on labeled data
    history = model.fit(x, y_train_one_hot,epochs=1,batch_size=32,validation_split=0.1,callbacks=[checkpoint_callback])
    
    # Open a GradientTape context to record operations for automatic differentiation
    with tf.GradientTape() as tape:
        # Compute predictions for labeled data
        predictions = model(x)
        # Compute original loss
        loss_value = loss_fn(y_train_one_hot, predictions)
        
        # Generate four different views for each tweet in the unlabeled data by droping words
        unlabeled_sequences_views = []
        for sequence in X_unlabeled:
            unlabeled_sequences_views.append(sequences)
            for _ in range(3):
                cross_view = sequence.copy()  # Make a copy of the original sequence
                drop = np.random.choice(64, size=1, replace=False)[0]
                cross_view[drop] = drop_symbol
                unlabeled_sequences_views.append(cross_view)


        # Compute predictions for unlabeled data using the model
        unlabeled_preds = model.predict(unlabeled_sequences_views)


        # Repeat the target labels to match the number of views for each sample
        unlabeled_labels_repeated = tf.repeat(unlabeled_labels, repeats=4, axis=0)

        # Compute consistency loss between predictions from different views
        consistency_loss_value = tf.reduce_mean(tf.abs(unlabeled_preds[::4] - unlabeled_preds[1::4])) * consistency_lambda

        # Compute total loss
        total_loss = loss_value + consistency_loss_value

    # Compute gradients
    gradients = tape.gradient(total_loss, model.trainable_variables)

    # Update optimizer's parameters based on gradients
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    print(f'Epoch [{epoch + 1}/{num_epochs}], Consistency Loss: {consistency_loss_value.numpy():.4f}')


In [63]:
model.load_weights('best_weights.keras')

In [ ]:
x.shape

In [64]:
model.save('sup_tweets200k_v2.h5')

ValueError: Unable to synchronously create dataset (name already exists)

In [74]:
# lef 3al sentences we we for each word 7ot 1 fel index beta3 el word
# we make adjustments for the decoder target data
ex_length = 100000
max_len = max([ len(sentence.split()) for sentence in sentences ])
encoder_input_data = np.zeros([ex_length,max_len+5,100])
# decoder_output_data = np.zeros([ex_length,max_len+5,100])
# decoder_input_data = np.zeros([ex_length,max_len+5,100])

In [ ]:
test_set = list(sentences[200000:].astype(str))
ex_length = len(test_set)
encoder_input_data = np.zeros([ex_length,max_len+5,100])
for ex in range(ex_length):
    words = test_set[ex].split()
    position = 0
    for word in words:
        if word in sg_model.wv:
            encoder_input_data[ex,position,:] = sg_model.wv[word]
        position += 1       